In [1]:
!pip -V

pip 23.3.1 from /opt/conda/lib/python3.10/site-packages/pip (python 3.10)


In [2]:
!pip list | grep scikit-learn

scikit-learn              1.5.1


In [8]:
# import cv2
from sklearn.feature_extraction import image as skimage
from sklearn.cluster import spectral_clustering
from PIL import Image
import numpy as np
from sklearn.cluster import KMeans, SpectralClustering
import matplotlib.pyplot as plt


def kmeans_color_quantization(image, k=3):
    # Преобразование изображения в формат, подходящий для KMeans
    pixel_values = image.reshape((-1, 3))
    pixel_values = np.float32(pixel_values)

    # Параметры для KMeans
    # criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
    kmeans = KMeans(n_clusters=k, random_state=42)

    # Применение KMeans
    labels = kmeans.fit_predict(pixel_values)
    centers = kmeans.cluster_centers_

    # Преобразование центров обратно в uint8
    centers = np.uint8(centers)

    # Назначение каждого пикселя изображения цветом соответствующего кластера
    segmented_image = centers[labels.flatten()]
    segmented_image = segmented_image.reshape(image.shape)

    return segmented_image



def spectral_color_quantization(image, k=3):
    mask = image.astype(bool)
    img = image.astype(float)
    graph = skimage.img_to_graph(img, mask=mask)
    graph.data = np.exp(-graph.data / graph.data.std()) # TODO
    print(f'{graph.data.shape=}')


    labels = spectral_clustering(graph, n_clusters=k, eigen_solver="arpack")
    label_im = np.full(mask.shape, -1.0)
    label_im[mask] = labels

    return label_im
    # # Преобразование изображения в формат, подходящий для SpectralClustering
    # pixel_values = image.reshape((-1, 3))
    # pixel_values = np.float32(pixel_values)

    # # Удаление дубликатов из features
    # unique_features, indices = np.unique(pixel_values, axis=0, return_inverse=True)
    # print(f'{pixel_values.shape=} {unique_features.shape=}')



    # # Параметры для SpectralClustering
    # sc = SpectralClustering(n_clusters=k, eigen_solver='lobpcg',
    #                         assign_labels="kmeans", 
    #                         affinity='cosine',
    #                         random_state=42, n_jobs=-1)
    # sc.fit(unique_features)

    # # Применение SpectralClustering
    # # labels = sc.fit_predict(unique_features)
    # centers = sc.cluster_centers_
    # labels = sc.labels_


    # # Преобразование центров обратно в uint8
    # centers = np.uint8(centers)

    # # Назначение каждого пикселя изображения цветом соответствующего кластера
    # segmented_image = centers[labels.flatten()]
    # segmented_image = segmented_image.reshape(image.shape)

    # return segmented_image

In [9]:
# Загрузка изображения
# image = cv2.imread('path_to_your_image.jpg')
# image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Преобразование в RGB
# image = Image.open('../assets/DJI_1065.png').convert('RGB')
image = Image.open(
    '../assets/UT_cytopathology_213_239_roi_7333.jpeg').convert('RGB')
# image = Image.open('../assets/c16-part1.png').convert('RGB')


# Загрузка изображения
# image = cv2.imread('path_to_your_image.jpg')
# image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Преобразование в RGB
# image = Image.open('../assets/DJI_1065.png').convert('RGB')
image = Image.open(
    '../assets/UT_cytopathology_213_239_roi_7333.jpeg').convert('RGB')
# image = Image.open('../assets/c16-part1.png').convert('RGB')

In [15]:
img = np.array(image)

mask = img.astype(bool)
img = img.astype(float)
graph = skimage.img_to_graph(img, mask=mask)
graph.data = np.exp(-graph.data / graph.data.std()) # TODO
print(f'{img.shape=} {graph.data.shape=}')
graph

Pimg.shape=(744, 744, 3) graph.data.shape=(10508195,)


<COOrdinate sparse matrix of dtype 'float64'
	with 10508195 stored elements and shape (1660603, 1660603)>

In [17]:
img.shape

(744, 744, 3)

In [16]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import coo_matrix

def create_affinity_matrix(img):
    w, h, _ = img.shape
    # Построение матрицы смежности
    n_nodes = w * h
    adjacency_matrix = np.zeros((n_nodes, n_nodes))

    for i in range(w):
        for j in range(h):
            node_index = i * h + j
            neighbors = [
                (i-1, j-1), (i-1, j), (i-1, j+1),
                (i, j-1),             (i, j+1),
                (i+1, j-1), (i+1, j), (i+1, j+1)
            ]
            for ni, nj in neighbors:
                if 0 <= ni < w and 0 <= nj < h:
                    neighbor_index = ni * h + nj
                    adjacency_matrix[node_index, neighbor_index] = 1

    # Вычисление косинусного подобия
    similarity_matrix = np.zeros_like(adjacency_matrix)

    for i in range(n_nodes):
        for j in range(n_nodes):
            if adjacency_matrix[i, j] == 1:
                similarity_matrix[i, j] = cosine_similarity(img[i][j], [vectors[j]])[0, 0]

    # Преобразование в разряженную матрицу
    return coo_matrix(similarity_matrix)


(1660603, 1660603)

In [4]:

from joblib import parallel_backend

with parallel_backend('threading', n_jobs=10):
    # Применение K-means кластеризации
    k = 3  # Количество кластеров
    segmented_image = spectral_color_quantization(np.array(image), k)

# Отображение результата
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.imshow(image)
plt.title('Original Image')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(segmented_image)
plt.title(f'Spectral Segmented Image with K={k}')
plt.axis('off')

plt.show()


graph.data.shape=(10508195,)


KeyboardInterrupt: 

In [ ]:
from PIL import Image
import numpy as np
from sklearn.cluster import KMeans
import os


def kmeans_color_quantization(pixel_values, k=3):
    # Параметры для KMeans
    kmeans = KMeans(n_clusters=k, random_state=42)

    # Применение KMeans
    kmeans.fit(pixel_values)

    # Преобразование центров обратно в uint8
    centers = np.uint8(kmeans.cluster_centers_)

    return kmeans, centers


# Путь к папке с изображениями
image_folder = 'path_to_your_image_folder'
output_folder = 'path_to_output_folder'

# Создание выходной папки, если она не существует
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Количество кластеров
k = 3

# Список для хранения всех пикселей
all_pixel_values = []

# Обработка каждого изображения в папке
for filename in os.listdir(image_folder):
    if filename.endswith(('.png', '.jpg', '.jpeg')):
        # Загрузка изображения
        image_path = os.path.join(image_folder, filename)
        image = Image.open(image_path).convert('RGB')

        # Преобразование изображения в формат, подходящий для KMeans
        pixel_values = np.array(image).reshape((-1, 3))
        pixel_values = np.float32(pixel_values)

        # Добавление пикселей в общий список
        all_pixel_values.append(pixel_values)

# Объединение всех пикселей в один большой набор данных
all_pixel_values = np.vstack(all_pixel_values)

# Применение K-means кластеризации ко всему набору данных
kmeans, centers = kmeans_color_quantization(all_pixel_values, k)

# Обработка каждого изображения в папке с использованием полученных центров кластеров
for filename in os.listdir(image_folder):
    if filename.endswith(('.png', '.jpg', '.jpeg')):
        # Загрузка изображения
        image_path = os.path.join(image_folder, filename)
        image = Image.open(image_path).convert('RGB')

        # Преобразование изображения в формат, подходящий для KMeans
        pixel_values = np.array(image).reshape((-1, 3))
        pixel_values = np.float32(pixel_values)

        # Назначение каждого пикселя изображения цветом соответствующего кластера
        labels = kmeans.predict(pixel_values)
        segmented_image = centers[labels.flatten()]
        segmented_image = segmented_image.reshape(image.shape)

        # Сохранение сегментированного изображения
        output_path = os.path.join(output_folder, filename)
        segmented_image = Image.fromarray(segmented_image.astype('uint8'))
        segmented_image.save(output_path)

        print(f'Processed and saved: {output_path}')

print('All images processed and saved.')